# Install Intel Distribution of OpenVINO Toolkit
Run the below cell to install OpenVino Directly to Colab (Takes 1 min)

In [ ]:
!wget https://apt.repos.intel.com/openvino/2020/GPG-PUB-KEY-INTEL-OPENVINO-2020
!apt-key add GPG-PUB-KEY-INTEL-OPENVINO-2020
!touch /etc/apt/sources.list.d/intel-openvino-2020.list
!echo "deb https://apt.repos.intel.com/openvino/2020 all main" >> /etc/apt/sources.list.d/intel-openvino-2020.list

!apt update
!apt install intel-openvino-dev-ubuntu18-2020.4.287

!pip install test-generator==0.1.1
!source /opt/intel/openvino/bin/setupvars.sh

# Download and Extract Dataset and Models

In [ ]:
!pip install gdown
!gdown --id 12RPFKynXFu_A8ETAuwti_Hq4yXXz0GCr
!gdown --id 1sLTFb4Rcv5HnlyE2iLFEOKqDbK8vbJ1s
!unzip "/content/state-farm-distracted-driver-detection.zip"
!unzip "/content/models.zip"

# Importing Required Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import glob
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense
from keras.optimizers import Adam
from keras.losses import CategoricalCrossentropy
from keras.utils.vis_utils import plot_model

# Preprocessing Data

In [3]:
def _prepareData(path): 
    imgsList = []
    labels = []
    for directory in sorted(glob.glob(os.path.join(path, '*')), key = lambda k: k.split("/")[-1]):
            for imgs in glob.glob(os.path.join(directory,'*.jpg')):
                img_cv = cv2.imread(imgs)
                img_cv_r = cv2.resize(img_cv,(128,128))
                imgsList.append(img_cv_r)
                labels.append(int(directory.split("/")[-1].replace('c','')))
    
    X_Train, X_Test, Y_Train, Y_Test =  train_test_split(imgsList,labels, test_size = 0.2)
    Y_Train = tf.keras.utils.to_categorical(Y_Train, num_classes=10)
    Y_Test = tf.keras.utils.to_categorical(Y_Test, num_classes=10)

    return np.array(X_Train), np.array(X_Test), Y_Train, Y_Test

#Paths
pathTrain_Images = "/content/imgs/train/"
pathPropagate_Images =  "/content/imgs/test/"

#List of Images for Train and Test
X_Train, X_Test, Y_Train, Y_Test = _prepareData(pathTrain_Images)

print("Size X_Train: {}, Size Y_Train: {}".format(len(X_Train),len(Y_Train)))
print("Size X_Test: {}, Size Y_Test: {}".format(len(X_Test),len(Y_Test)))

Size X_Train: 17939, Size Y_Train: 17939
Size X_Test: 4485, Size Y_Test: 4485


# Defining Model

In [5]:
model = keras.models.Sequential()

model.add(keras.layers.InputLayer(input_shape=(128, 128, 3)))
model.add(keras.layers.Conv2D(filters=32,kernel_size=(5,5),strides = (1,1),padding='same',activation='relu',name='Conv_1'))
model.add(keras.layers.MaxPool2D(pool_size = (2,2),name = 'Pool_1'))
model.add(keras.layers.Conv2D(filters = 64,kernel_size = (5,5),strides = (1,1),padding = 'same',activation = 'relu',name = 'Conv_2'))
model.add(keras.layers.MaxPool2D(pool_size = (2,2),name = 'Pool_2'))
model.add(keras.layers.Conv2D(filters = 128,kernel_size = (5,5),strides = (1,1),padding = 'same',activation = 'relu',name = 'Conv_3'))
model.add(keras.layers.MaxPool2D(pool_size = (2,2),name = 'Pool_3'))
model.add(keras.layers.Conv2D(filters = 256,kernel_size = (5,5),strides = (1,1),padding = 'same',activation = 'relu',name = 'Conv_4'))
model.add(keras.layers.MaxPool2D(pool_size = (2,2),name = 'Pool_4'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(units=1024, activation='relu',name = 'fc_1'))
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(units=512, activation='relu',name = 'fc_2'))
model.add(keras.layers.Dense(units=10,activation='softmax',name = 'fc_3'))

tf.random.set_seed(1)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(from_logits = False), metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv_1 (Conv2D)              (None, 128, 128, 32)      2432      
_________________________________________________________________
Pool_1 (MaxPooling2D)        (None, 64, 64, 32)        0         
_________________________________________________________________
Conv_2 (Conv2D)              (None, 64, 64, 64)        51264     
_________________________________________________________________
Pool_2 (MaxPooling2D)        (None, 32, 32, 64)        0         
_________________________________________________________________
Conv_3 (Conv2D)              (None, 32, 32, 128)       204928    
_________________________________________________________________
Pool_3 (MaxPooling2D)        (None, 16, 16, 128)       0         
_________________________________________________________________
Conv_4 (Conv2D)              (None, 16, 16, 256)       8

# Training Model

**It takes 30 mins to train Model,you can also skip this step and continue to inferencing as I have already trained and downloaded that model**

In [ ]:
history = model.fit(x = X_Train, y=Y_Train,epochs = 10, batch_size = 500, verbose = 1,validation_split=0.2)
model.save("/content/models/driver_distraction.h5")
test_loss, test_acc = model.evaluate(X_Test, Y_Test, verbose = 1)

# Inferencing Model (Normal)

It Evaluates first 50 Images from Test Dataset 

Average Time per Image : 0.0551 Seconds.

In [ ]:
import time
model.load_weights('/content/models/driver_distraction.h5')
df = pd.DataFrame({'img':[],'c0':[], 'c1':[],'c2':[], 'c3':[], 'c4':[],'c5':[], 'c6':[], 'c7':[], 'c8':[], 'c9':[]})
def submission(pathPropagate_Images,df):
    cnt=0
    actions=["Safe Driving","Texting Right Hand","Talking on Phone Right Hand","Texting Left Hand","Talking on Phone Left Hand","Operating on Radio","Drinking","Reaching Behind","Hair and Makeup","Talking to Passenger"]

    for imgs in glob.glob(os.path.join(pathPropagate_Images,'*.jpg')):
        img_cv = cv2.imread(imgs)
        img_cv_r = cv2.resize(img_cv,(128,128))
        img_cv_predict = np.reshape(img_cv_r,[1,128,128,3])
        arr_predict = model.predict(img_cv_predict,batch_size = 1)
        arr=arr_predict[0].tolist()
        mx=max(arr)
        ind=arr.index(mx)
        print(imgs,actions[ind])
        cnt+=1
        if(cnt>50):
          break
start_time = time.time()
pathPropagate_Images =  "/content/imgs/test/"
submission(pathPropagate_Images,df)
print("Total Time: ", time.time()-start_time)
print("Average Time Per Image: ", (time.time()-start_time)/50)

# Inferencing using OpenVino

### Step 1 : Freezing Models

In [ ]:
from tensorflow.python.framework import graph_io
from tensorflow.keras.models import load_model
import tensorflow.compat.v1 as tf
#Using Tensorflow 1.0 due to Compatibility issues
tf.disable_v2_behavior()

# Clear any previous session.
tf.keras.backend.clear_session()

#Directory for Saving Model
save_pb_dir = '/content/models'
model_fname = '/content/models/driver_distraction.h5'

def freeze_graph(graph, session, output, save_pb_dir='/content/models', save_pb_name='frozen_model.pb', save_pb_as_text=False):
    with graph.as_default():
        graphdef_inf = tf.compat.v1.graph_util.remove_training_nodes(graph.as_graph_def())
        graphdef_frozen = tf.compat.v1.graph_util.convert_variables_to_constants(session, graphdef_inf, output)
        graph_io.write_graph(graphdef_frozen, save_pb_dir, save_pb_name, as_text=save_pb_as_text)
        return graphdef_frozen


tf.keras.backend.set_learning_phase(0) 

model = load_model(model_fname)
session=tf.keras.backend.get_session()
INPUT_NODE = [t.op.name for t in model.inputs]
OUTPUT_NODE = [t.op.name for t in model.outputs]
print(INPUT_NODE, OUTPUT_NODE)
frozen_graph = freeze_graph(session.graph, session, [out.op.name for out in model.outputs], save_pb_dir=save_pb_dir)

### Step 2 : Converted Frozen Model to BIN and XML format

In [ ]:
mo_tf_path = '/opt/intel/openvino/deployment_tools/model_optimizer/mo_tf.py'

pb_file = '/content/models/frozen_model.pb'
output_dir = '/content/models'
img_height = 128
input_shape = [1, img_height, img_height, 3]
input_shape_str = str(input_shape).replace(' ', '')

#Running Commands to run OPENVINO's Model Optimiser Converter 'mo_tf.py'
!source /opt/intel/openvino/bin/setupvars.sh
!python {mo_tf_path} --input_model {pb_file} --output_dir {output_dir} --input_shape {input_shape_str} --data_type FP16

### Step 3 : Running Inference (OpenVino)

This is evaluated on first 50 images from Testing Data from Dataset

Average Time per Image : 0.0258 Seconds

**OpenVino Inference is working 2.1 Times Faster than Normal Inference**

In [ ]:
!source /opt/intel/openvino/bin/setupvars.sh && \
python /content/models/inference.py

### Optional : Running OpenVino Inference on your Own Images

First Upload multiple jpg files and it will automatically give Output

In [ ]:
print("Upload JPG Images to be inferred")
files.upload()
!source /opt/intel/openvino/bin/setupvars.sh && \
python /content/models/inference_uploaded_images.py